Using Interactive Brokers API to get data. Only focusing on data pulling, as there are multiple avenues that can be taken for different types of data being pulled (stock, options, historical, live, etc.). All data processing located elsewhere

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta, time, datetime


import warnings
from IPython.core.display import display, HTML
warnings.filterwarnings("ignore") # suppress warnings because theyre annoying and i like living on the edge
pd.set_option('display.max_columns',100) # so i can see all columns
np.set_printoptions(linewidth = 220) # increase numpy array print
display(HTML("<style>.container { width:80% !important; }</style>")) # widen jupyter notebook blocks

In [2]:

from ibapi.wrapper import EWrapper
from ibapi.client import EClient
from ibapi.contract import Contract as IBcontract
from threading import Thread
import queue
import datetime

DEFAULT_HISTORIC_DATA_ID=50
DEFAULT_GET_CONTRACT_ID=43

## marker for when queue is finished
FINISHED = object()
STARTED = object()
TIME_OUT = object()

class finishableQueue(object):

    def __init__(self, queue_to_finish):

        self._queue = queue_to_finish
        self.status = STARTED

    def get(self, timeout):
        """
        Returns a list of queue elements once timeout is finished, or a FINISHED flag is received in the queue
        :param timeout: how long to wait before giving up
        :return: list of queue elements
        """
        contents_of_queue=[]
        finished=False

        while not finished:
            try:
                current_element = self._queue.get(timeout=timeout)
                if current_element is FINISHED:
                    finished = True
                    self.status = FINISHED
                else:
                    contents_of_queue.append(current_element)
                    ## keep going and try and get more data

            except queue.Empty:
                ## If we hit a time out it's most probable we're not getting a finished element any time soon
                ## give up and return what we have
                finished = True
                self.status = TIME_OUT


        return contents_of_queue

    def timed_out(self):
        return self.status is TIME_OUT





class TestWrapper(EWrapper):
    """
    The wrapper deals with the action coming back from the IB gateway or TWS instance
    We override methods in EWrapper that will get called when this action happens, like currentTime
    Extra methods are added as we need to store the results in this object
    """

    def __init__(self):
        self._my_contract_details = {}
        self._my_historic_data_dict = {}

    ## error handling code
    def init_error(self):
        error_queue=queue.Queue()
        self._my_errors = error_queue

    def get_error(self, timeout=5):
        if self.is_error():
            try:
                return self._my_errors.get(timeout=timeout)
            except queue.Empty:
                return None

        return None

    def is_error(self):
        an_error_if=not self._my_errors.empty()
        return an_error_if

    def error(self, id, errorCode, errorString):
        ## Overriden method
        errormsg = "IB error id %d errorcode %d string %s" % (id, errorCode, errorString)
        self._my_errors.put(errormsg)


    ## get contract details code
    def init_contractdetails(self, reqId):
        contract_details_queue = self._my_contract_details[reqId] = queue.Queue()

        return contract_details_queue

    def contractDetails(self, reqId, contractDetails):
        ## overridden method

        if reqId not in self._my_contract_details.keys():
            self.init_contractdetails(reqId)

        self._my_contract_details[reqId].put(contractDetails)

    def contractDetailsEnd(self, reqId):
        ## overriden method
        if reqId not in self._my_contract_details.keys():
            self.init_contractdetails(reqId)

        self._my_contract_details[reqId].put(FINISHED)

    ## Historic data code
    def init_historicprices(self, tickerid):
        historic_data_queue = self._my_historic_data_dict[tickerid] = queue.Queue()

        return historic_data_queue


    def historicalData(self, tickerid , bar):

        ## Overriden method
        ## Note I'm choosing to ignore barCount, WAP and hasGaps but you could use them if you like
        bardata=(bar.date, bar.open, bar.high, bar.low, bar.close, bar.volume)

        historic_data_dict=self._my_historic_data_dict

        ## Add on to the current data
        if tickerid not in historic_data_dict.keys():
            self.init_historicprices(tickerid)

        historic_data_dict[tickerid].put(bardata)

    def historicalDataEnd(self, tickerid, start:str, end:str):
        ## overriden method

        if tickerid not in self._my_historic_data_dict.keys():
            self.init_historicprices(tickerid)

        self._my_historic_data_dict[tickerid].put(FINISHED)




class TestClient(EClient):
    """
    The client method
    We don't override native methods, but instead call them from our own wrappers
    """
    def __init__(self, wrapper):
        ## Set up with a wrapper inside
        EClient.__init__(self, wrapper)


    def resolve_ib_contract(self, ibcontract, reqId=DEFAULT_GET_CONTRACT_ID):

        """
        From a partially formed contract, returns a fully fledged version
        :returns fully resolved IB contract
        """

        ## Make a place to store the data we're going to return
        contract_details_queue = finishableQueue(self.init_contractdetails(reqId))

        print("Getting full contract details from the server... ")

        self.reqContractDetails(reqId, ibcontract)

        ## Run until we get a valid contract(s) or get bored waiting
        MAX_WAIT_SECONDS = 30
        new_contract_details = contract_details_queue.get(timeout = MAX_WAIT_SECONDS)

        while self.wrapper.is_error():
            print(self.get_error())

        if contract_details_queue.timed_out():
            print("Exceeded maximum wait for wrapper to confirm finished - seems to be normal behaviour")

        if len(new_contract_details)==0:
            print("Failed to get additional contract details: returning unresolved contract")
            return ibcontract

        if len(new_contract_details)>1:
            print("got multiple contracts using first one")

        new_contract_details=new_contract_details[0]

        resolved_ibcontract=new_contract_details.contract

        return resolved_ibcontract


    def get_IB_historical_data(self, ibcontract, durationStr="1 D", barSizeSetting="5 secs",
                               tickerid=DEFAULT_HISTORIC_DATA_ID):

        """
        Returns historical prices for a contract, up to today
        ibcontract is a Contract
        :returns list of prices in 4 tuples: Open high low close volume
        """


        ## Make a place to store the data we're going to return
        historic_data_queue = finishableQueue(self.init_historicprices(tickerid))

        # Request some historical data. Native method in EClient
        self.reqHistoricalData(
            tickerid,  # tickerId,
            ibcontract,  # contract,
            datetime.datetime.today().strftime("%Y%m%d %H:%M:%S %Z"),  # endDateTime,
            durationStr,  # durationStr,
            barSizeSetting,  # barSizeSetting,
            "TRADES",  # whatToShow,
            1,  # useRTH,
            1,  # formatDate
            False,  # KeepUpToDate <<==== added for api 9.73.2
            [] ## chartoptions not used
        )



        ## Wait until we get a completed data, an error, or get bored waiting
        MAX_WAIT_SECONDS = 200
        print("Getting historical data from the server... could take %d seconds to complete " % MAX_WAIT_SECONDS)

        historic_data = historic_data_queue.get(timeout = MAX_WAIT_SECONDS)

        while self.wrapper.is_error():
            print(self.get_error())

        if historic_data_queue.timed_out():
            print("Exceeded maximum wait for wrapper to confirm finished - seems to be normal behaviour")

        self.cancelHistoricalData(tickerid)


        return historic_data



class TestApp(TestWrapper, TestClient):
    def __init__(self, ipaddress, portid, clientid):
        error_queue=queue.Queue()
        self._my_errors = error_queue
        
        TestWrapper.__init__(self)
        TestClient.__init__(self, wrapper=self)

        self.connect(ipaddress, portid, clientid)

        thread = Thread(target = self.run)
        thread.start()

        setattr(self, "_thread", thread)

        self.init_error()


#if __name__ == '__main__':

app = TestApp("127.0.0.1", 7497, 1)

ibcontract = IBcontract()
ibcontract.secType = "STK"
ibcontract.lastTradeDateOrContractMonth="202010"
ibcontract.symbol="AAPL"
ibcontract.exchange="SMART"

resolved_ibcontract=app.resolve_ib_contract(ibcontract)

historic_data = app.get_IB_historical_data(resolved_ibcontract)

# print(historic_data)

# app.disconnect()

Getting full contract details from the server... 
IB error id -1 errorcode 2104 string Market data farm connection is OK:usfarm.nj
IB error id -1 errorcode 2104 string Market data farm connection is OK:usfarm
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:euhmds
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:fundfarm
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:ushmds
IB error id -1 errorcode 2158 string Sec-def data farm connection is OK:secdefil
got multiple contracts using first one
Getting historical data from the server... could take 200 seconds to complete 


In [3]:
historic_data

[('20201002  06:30:00', 112.84, 112.93, 112.5, 112.5, 25395),
 ('20201002  06:30:05', 112.3, 112.75, 112.22, 112.61, 3029),
 ('20201002  06:30:10', 112.61, 112.9, 112.23, 112.9, 1516),
 ('20201002  06:30:15', 112.9, 113.03, 112.39, 113.0, 1823),
 ('20201002  06:30:20', 112.97, 113.11, 112.73, 113.11, 1557),
 ('20201002  06:30:25', 113.11, 113.25, 113.05, 113.25, 1219),
 ('20201002  06:30:30', 113.22, 113.27, 113.06, 113.18, 985),
 ('20201002  06:30:35', 113.19, 113.29, 113.15, 113.28, 824),
 ('20201002  06:30:40', 113.28, 113.46, 113.23, 113.44, 1010),
 ('20201002  06:30:45', 113.44, 113.46, 113.37, 113.38, 1015),
 ('20201002  06:30:50', 113.4, 113.5, 113.33, 113.48, 758),
 ('20201002  06:30:55', 113.47, 113.58, 113.45, 113.58, 1346),
 ('20201002  06:31:00', 113.58, 113.78, 113.55, 113.77, 860),
 ('20201002  06:31:05', 113.75, 113.79, 113.58, 113.6, 710),
 ('20201002  06:31:10', 113.63, 113.67, 113.57, 113.63, 875),
 ('20201002  06:31:15', 113.64, 113.66, 113.59, 113.65, 459),
 ('20201

In [4]:
import pickle

# save csv for tableau or otherwise
work_dir = 'D:/general_data/stock'
with open(work_dir + 'aapl_20201002.pkl','wb') as pickle_out:
    pickle.dump(historic_data, pickle_out)
# df.to_csv(work_dir + '/aapl_20200928.csv', index = False)